In [1]:
%load_ext autoreload
%autoreload 2

import torch
torch.set_default_dtype(torch.float64)

import numpy as np
np.set_printoptions(2, suppress=True)
torch.set_printoptions(2, sci_mode=False)

import sys
sys.path.append('../')
from panda_kinematics import PandaKinematics
from ttgo import TTGO
from manipulator_utils import exp_space, test_robotics_task
from utils import Point2PointMotion, test_ttgo
from panda_cost_utils import PandaCost,SDF_Cost 
device = "cpu" #torch.device("cuda" if torch.cuda.is_available() else "cpu")from motion_generation_utils import Point2PointMotion,Point2PointMotionCostFcn

In [2]:
# import the environment (SDF and for graphics visualization in pybullet)
import sys
DATA_PATH = './data'
body_sphere_path = './data/sphere_setting.npy'
sdf_path = './data/sdf.npy'
urdf_path = './data/urdf/frankaemika_new/panda_arm.urdf'

In [3]:
file_name = 'panda-reach-mp-0-b-0.05-0.05-0.05-0.25-1.5-d0_theta-30-d0_w-30-nswp-30-rmax-100-kr-3.0-basis-rbf-margin-0.1-dt-0.1.pickle'

Load the model and the parameters

In [4]:
# Load the parameters from the training
model = torch.load(file_name,map_location=torch.device("cpu"))
tt_model = model['tt_model']
w_goal,w_obst,w_orient, w_ee, w_control = model['w']
b_goal,b_obst,b_orient, b_ee, b_control = model['b']

print("w: ",w_goal,w_obst,w_orient, w_ee, w_control)
print("b: ",b_goal,b_obst,b_orient, b_ee, b_control)

d0_theta, d0_w = model['d0']
K = model['K']
margin= model['margin']
domain, domain_task, domain_x_shelf, domain_x_box = model['domains']
theta_0 = model['theta_0']
theta_2 = model['theta_2']
theta_3 = model['theta_3']
dt = 0.01 #model['dt'] #
basis = model['basis']
Rd_0= model['Rd_0']
v_d= model['v_d']
mp = model['mp']
test_task = model['test_task'] #torch.load('test_target.pickle')#
key_points_weight = model['key_points_weight']
key_points_margin = model['key_points_margin']
panda = model['panda'];panda.set_device('cpu')
p2p = model['p2p']; p2p.set_device('cpu')
pandaCost = model['pandaCost']; pandaCost.set_device('cpu')
sdf_cost = model['sdf_cost']; sdf_cost.set_device('cpu')

theta_bounds = [panda.min_config, panda.max_config]
p2p = Point2PointMotion(dt=dt, K=K, n=7, bounds=theta_bounds, basis=basis, device=device)
pandaCost = PandaCost(p2p_motion=p2p,robot=panda, sdf_cost=sdf_cost, 
                    Rd_0=Rd_0, v_d=v_d,
                    w_ee=w_ee, w_control=w_control, w_goal=w_goal, w_obst=w_obst, w_orient=w_orient,
                    b_obst=b_obst,b_goal=b_goal,
                    b_control=b_control, b_orient=b_orient, b_ee=b_ee,
                    device=device)  


data_sdf = np.load('./data/sdf.npy', allow_pickle=True)[()]
sdf_matr = data_sdf['sdf_matr']  # SDF tensor
bounds = torch.tensor(data_sdf['bounds']).to(device) # Bound of the environment
env_bound = data_sdf['env_bound']  
shelf_bound = data_sdf['shelf_bound'] 
box_bound = data_sdf['box_bound'] 


w:  0.25 0.25 0.25 0.0 0.25
b:  0.05 0.05 0.05 1.5 0.25


Cost functions

In [5]:
if mp == 0: # target-reaching-from-fixed-initial-configuration
    print("Target reaching from a fixed initial configuration")
    def cost_all(x):
        return pandaCost.cost_j2p(x,theta_0)

elif mp == 1: # via-point problem between fixed initial and final configurations
    print("Via-point-1 problem between fixed initial and final configurations")
    def cost_all(x):
        return pandaCost.cost_j2p2j(x, theta_0, theta_2)

elif mp == 2: # fixed initial, given two target points to reach in sequence
    print("Reach two target points in sequence")
    def cost_all(x):
        return pandaCost.cost_j2p2p(x, theta_0)

elif mp == 3: # fixed initial and , given two target points to reach in sequence
    print("Via-point-2 problem between fixed initial and final configurations")
    def cost_all(x):
        return pandaCost.cost_j2p2p2j(x, theta_0, theta_3)


print("Discretization: ",[len(x) for x in domain])


def cost(x):
    return cost_all(x)[:,0]

def pdf(x):
    return torch.exp(-cost(x)**2)



Target reaching from a fixed initial configuration
Discretization:  [60, 44, 17, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30]


TT-Model

In [6]:
# Fit the TT-model
ttgo = TTGO(domain=domain, pdf=pdf,cost=cost,device=device)
ttgo.tt_model = tt_model
ttgo.round(1e-6)
ttgo.to('cpu')

# Prepare for the task
sites_task = list(range(3)) #list(range(len(domain_task)))
ttgo.set_sites(sites_task)

In [7]:
sites_task

[0, 1, 2]

Generate test set

In [8]:
# # generate test set for target point 
# ns = 50

# ns_ = ns
# test_x_1 = torch.zeros(ns_,3).to(device)
# test_x_2 = torch.zeros(ns_,3).to(device)
# for i in range(3):
#     unif = torch.distributions.uniform.Uniform(low=domain_x_shelf[i][0],high=domain_x_shelf[i][-1])
#     test_x_1[:,i]= torch.tensor([unif.sample() for i in range(ns_)]).to(device)
#     unif = torch.distributions.uniform.Uniform(low=domain_x_box[i][0],high=domain_x_box[i][-1])
#     test_x_2[:,i]= torch.tensor([unif.sample() for i in range(ns_)]).to(device)

# # test_x_1  = test_x_1[(sdf_cost.sdf(test_x_1)-0.09)>0][:ns] # 
# # test_x_2  = test_x_2[(sdf_cost.sdf(test_x_2)-0.09)>0][:ns] # 


# if mp==0 or mp==1: 
#     test_task = test_x_1
# if mp==2 or mp==3:
#     test_task = torch.cat((test_x_1, test_x_2),dim=-1)


Test the model

In [9]:
# norm=1
# print("total-cost | goal | collision | orientation | ee | control ")
# # for alpha in [0.99,0.9,0.8,0.5]:
# #     for n_samples_tt in [10,50,100,1000]:
# #         _ = test_ttgo(ttgo=ttgo.clone(), cost=cost_all, 
# #             test_task=test_task, n_samples_tt=n_samples_tt,
# #             alpha=alpha, norm=norm, device=device, test_rand=True)
# alpha = 0.75; norm=1; n_samples_tt = 1000
# costs_tt, costs_tt_opt, costs_rand, costs_rand_opt, tt_nit, rand_nit = test_ttgo(ttgo=ttgo.clone(), cost=cost_all, 
#             test_task=test_task, n_samples_tt=n_samples_tt,
#             alpha=alpha, norm=norm, device=device, test_rand=True)


In [10]:
# max_batch = 10000
# def cost_batched(x):
#     ''' To avoid memorry issues with large batch processing in tt-cross, reduce computation into smaller batches '''   
#     batch_size = x.shape[0]
#     cost_values = torch.zeros(batch_size,cost_all(x[0].view(1,-1)).shape[1]).to(device)
#     num_batch = batch_size//max_batch
#     end_idx = 0
#     for i in range(num_batch):
#         start_idx = i*max_batch
#         end_idx = (i+1)*max_batch
#         cost_values[start_idx:end_idx,:] =cost_all(x[start_idx:end_idx])
#     if batch_size>end_idx:          
#         cost_values[end_idx:batch_size,:] = cost_all(x[end_idx:batch_size])
#     return cost_values

In [11]:
# torch.save(test_task,"test_target.pickle")

In [12]:
# sample_set = [1,10,100,1000]
# alphas = [0.9,0.75,0.5,0]
# cut_total=0.25
# with torch.no_grad():
#     test_robotics_task(ttgo.clone(), cost_all, test_task, alphas, sample_set, cut_total,device)

## Visualization

In [13]:
# p.disconnect()

In [14]:
import pybullet_data
from panda_visualization_utils import *
import pybullet as p
from functools import partial

p.connect(p.GUI)
data = np.load(sdf_path, allow_pickle=True)[()]
sdf_matr = data['sdf_matr']  #SDF tensor
obstacles = data['obstacles'] #obstacles parameters
colors = [[0.8, 0.5, 0.5, 1]]*len(obstacles)
obj_id, init_id, target_id, border_id, obstacle_ids = init_pybullet (np.zeros(3), np.zeros(3), obstacles, colors=colors)
p.setPhysicsEngineParameter(enableFileCaching=0)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.configureDebugVisualizer(p.COV_ENABLE_GUI,0)
robot_urdf ='./data/urdf/frankaemika_new/panda_arm.urdf'
robot_id = p.loadURDF(fileName=robot_urdf)
dof = p.getNumJoints(robot_id)
pb_joint_indices = np.arange(7)
joint_limits = get_joint_limits(robot_id,pb_joint_indices)
mean_pose = 0.5*(joint_limits[0]+joint_limits[1])
set_q_std = partial(set_q,robot_id, pb_joint_indices)
rmodel = pin.buildModelFromUrdf(robot_urdf)
rdata = rmodel.createData()
pin_frame_names = [f.name for f in rmodel.frames]
ee_frame_id = rmodel.getFrameId('panda_hand_joint')
alpha = np.deg2rad(52)
quat = p.getQuaternionFromAxisAngle((0,0,1),alpha)
p.resetBasePositionAndOrientation(robot_id, (0,0,0.05), quat)

For a random task find the decision variables (multiple solutions)

In [34]:
s = np.random.randint(0,test_task.shape[0]-1)
sample_xe = test_task[s][:3]
sample_xe[0] = -0.6
# sample_xe[:3]= torch.tensor([-0.55,0.2,0.15])
# sample_xe[0]=-0.7; sample_xe[2]=0.1; sample_xe[5]=0.3; sample_xe[4]=0.65; sample_xe[3]=-0.15
# sample_xe[:3] = torch.tensor([0.55, 0.5, 0.75])
#torch.tensor([0.6, 0.4, 0.45]) #center center
#test_task[s]#torch.tensor([ 0.66,  0.16,  0.40, -0.17,  0.55,  0.32]) #torch.tensor([0.6, 0.47, 0.75, -0.25,  0.61,  0.35])#test_task[s]
# torch.tensor([ 0.66,  0.16,  0.40, -0.17,  0.55,  0.32]) # 
# tensor([ 0.66,  0.16,  0.40, -0.17,  0.55,  0.32])
# torch.tensor([0.54, 0.63, 0.12])
#torch.tensor([0.70, 0.40, 0.15])#
print(sample_xe)
# torch.tensor([0.55, 0.4347, 0.4]) # middle center
# torch.tensor([0.4435, 0.4347, 50.2139]) # bottom center
# torch.tensor([0.4991, 0.6301, 0.3695]) # bottom left
# torch.tensor([0.6352, 0.4708, 0.7535]) # top left
# torch.tensor([0.6789, 0.1950, 0.6976]) # top shelf center
# torch.tensor([0.6635, 0.6250, 0.2031]) # bottom, in, inaccessible
# tirch.tensor([0.4228, 0.4441, 0.8760]) # top 
# sample_xe = torch.tensor([0.8, -0.1, 0.3]) #torch.tensor([0.8, -0.3, 0.35])#torch.torch.tensor([ 0.6404, 0.2350,  0.549])#torch.tensor([ 0.7130, -0.3007, -0.1276]) #
n_solutions= 3
n_samples_tt = 1000
n_samples_rand= n_samples_tt

alpha=0.85; norm=1 ;

t1 = time.time()
samples_tt, samples_idx = ttgo.sample(n_samples=n_samples_tt, x_task=sample_xe.reshape(1,-1),alpha=alpha,norm=norm)
state_k_tt = ttgo.choose_top_k_sample(samples_tt,n_solutions)
t2=time.time()

#Optimize
state_k_tt_opt = 1*state_k_tt
for i, state in enumerate(state_k_tt):
    state_k_tt_opt[i,:],results= ttgo.optimize(state,bound=True)
t3 = time.time()
samples_rand, _ = ttgo.sample_random(n_samples=n_samples_rand,  x_task=sample_xe.reshape(1,-1))
state_k_rand = ttgo.choose_top_k_sample(samples_rand,n_solutions)
t4=time.time()

# #Optimize
state_k_rand_opt = state_k_rand*1
for i, state in enumerate(state_k_rand):
    state_k_rand_opt[i,:],results= ttgo.optimize(state,bound=True)
t5=time.time()

print("Time taken:", (t2-t1), t4-t3)
             
c_tt =  cost_all(state_k_tt_opt)
c_rand =   cost_all(state_k_rand_opt)

print("Cost-mean-tt:",torch.mean(c_tt,dim=0))
print("Cost-mean-rand:",torch.mean(c_rand,dim=0))



tensor([-0.60,  0.34,  0.09])
Time taken: 1.8695852756500244 1.0888481140136719
Cost-mean-tt: tensor([    0.36,     0.00,     0.00,     0.04,     3.65,     0.16])
Cost-mean-rand: tensor([    0.72,     0.02,     0.00,     0.01,     3.15,     0.53])


From the decision variables construct the joint angle trajectory

In [26]:
n_joints=7
theta_bounds = [panda.min_config, panda.max_config]
p2p_ =Point2PointMotion(dt=0.001, K=K, n=n_joints,bounds=theta_bounds, basis=basis, device=device)

x = 1*state_k_tt_opt
if mp==0:
    xgoal_1 = x[:,:3] # of the end-effector: batch x positions
    theta_1 = x[:,3:3+n_joints] # final configuration, batch x joint-angles
    w = x[:,3+n_joints:] # basis weights
    theta_t = p2p_.gen_traj_p2p(theta_0.repeat(n_solutions,1),theta_1,w) # batch x time x joint 

elif mp==1:
    xgoal_1 = x[:,:3] # via-point
    theta_1 = x[:,3:3+n_joints] # via-configuration
    w01 = x[:,3+n_joints:3+n_joints+n_joints*K] # basis weights for phase 1 (theta_0 to theta_1)
    w12 = x[:,3+n_joints+n_joints*K:] # basis weights for phase 2 (theta_1 to theta_2)
    # joint-angle traj from initial config to via config        
    theta_01_t = p2p_.gen_traj_p2p(theta_0.repeat(n_solutions,1),theta_1,w01) #batch x time/2 x joint
    # joint-angle traj from via-config to final config
    theta_12_t = p2p_.gen_traj_p2p(theta_1,theta_2.repeat(n_solutions,1),w12) #batch x time/2 x joint
    T01 = theta_01_t.shape[1]
    T12 = theta_12_t.shape[1]

elif mp==2:
    xgoal_1 = x[:,:3] # via-point-1
    x_goal_2 = x[:,3:6] # via-point-2
    theta_1 = x[:,6:6+n_joints] # via-configuration-1
    theta_2 = x[:,6+n_joints:6+2*n_joints] # via-configuration-1
    w01 = x[:,6+2*n_joints:6+2*n_joints+n_joints*K] # basis weights for phase 1 (theta_0 to theta_1)
    w12 = x[:,6+2*n_joints+n_joints*K:] # basis weights for phase 2 (theta_1 to theta_2)
    # joint-angle traj from initial config to via config        
    theta_01_t = p2p_.gen_traj_p2p(theta_0.repeat(n_solutions,1),theta_1,w01) #batch x time/2 x joint
    # joint-angle traj from via-config-1 to via-config-2 (final config)
    theta_12_t = p2p_.gen_traj_p2p(theta_1,theta_2,w12) #batch x time/2 x joint
    T01 = theta_01_t.shape[1]
    T12 = theta_12_t.shape[1]
    
elif mp==3:
    xgoal_1 = x[:,:3] # via-point-1
    x_goal_2 = x[:,3:6] # via-point-2
    theta_1 = x[:,6:6+n_joints] # via-configuration-1
    theta_2 = x[:,6+n_joints:6+2*n_joints] # via-configuration-1
    w01 = x[:,6+2*n_joints:6+2*n_joints+n_joints*K] # basis weights for phase 1 (theta_0 to theta_1)
    w12 = x[:,6+2*n_joints+n_joints*K:6+2*n_joints+2*n_joints*K] # basis weights for phase 2 (theta_1 to theta_2)
    w23 = x[:,6+2*n_joints+2*n_joints*K:]
    # joint-angle traj from initial config to via config        
    theta_01_t = p2p_.gen_traj_p2p(theta_0.repeat(n_solutions,1),theta_1,w01) #batch x time/2 x joint
    # joint-angle traj from via-config-1 to via-config-2
    theta_12_t = p2p_.gen_traj_p2p(theta_1,theta_2,w12) #batch x time/2 x joint
    # joint-angle traj from via-config-1 to via-config-2
    theta_23_t = p2p_.gen_traj_p2p(theta_2,theta_3.repeat(n_solutions,1),w23) #batch x time/2 x joint
    T01 = theta_01_t.shape[1]
    T12 = theta_12_t.shape[1]
    T23 = theta_23_t.shape[1]
    theta_t = torch.cat((theta_01_t,theta_12_t,theta_23_t),dim=1)

    
costs_ = cost(x)
print("Costs: ", costs_)


Costs:  tensor([0.00, 0.00, 0.74])


In [27]:
# print("Joint Violate: ", torch.sum(theta_t < panda.theta_min_robot) + torch.sum(theta_t > panda.theta_max_robot) )

In [28]:
_ , _,sphere_id = create_primitives(p.GEOM_SPHERE, radius = 0.02)
pos = x[0,:3]
p.resetBasePositionAndOrientation(sphere_id, pos, (0,0,0,1))


In [29]:
if mp>1:
    _ , _,sphere_id = create_primitives(p.GEOM_SPHERE, radius = 0.02)
    pos = x[0,3:6];
    p.resetBasePositionAndOrientation(sphere_id, pos, (0,0,0,1))


Visualize the joint angle trajectory

In [30]:
time.sleep(10)
dt = 0.01
if mp==0:
    for k_ in range(n_solutions):
        set_q_std(theta_t[0,0,:])
        T = theta_t.shape[1]
        time.sleep(30*dt)
        for t in range(T):
            set_q_std(theta_t[k_,t,:])
            time.sleep(1*dt)
        time.sleep(10*dt)
elif mp==1 or mp==2:
    for k_ in range(n_solutions):
        set_q_std(theta_01_t[0,0,:])
        time.sleep(2)
        for t in range(T01):
            set_q_std(theta_01_t[k_,t,:])
            time.sleep(1*dt)
        time.sleep(1)
        for t in range(T12):
            set_q_std(theta_12_t[k_,t,:])
            time.sleep(1*dt)
        time.sleep(2)
elif mp==3:
    for k_ in range(n_solutions):
        set_q_std(theta_01_t[0,0,:])
        time.sleep(2)
        for t in range(T01):
            set_q_std(theta_01_t[k_,t,:])
            time.sleep(1*dt)
        time.sleep(1)
        for t in range(T12):
            set_q_std(theta_12_t[k_,t,:])
            time.sleep(1*dt)
        time.sleep(1)
        for t in range(T23):
            set_q_std(theta_23_t[k_,t,:])
            time.sleep(1*dt)
    time.sleep(2)

In [35]:
# torch.save(theta_t,'reach_10.pickle')

In [22]:
b_control

0.25

In [23]:
b_ee

1.5